In [34]:
from itertools import chain
import pandas as pd
import numpy as np
from re import T
from sklearn.metrics import mean_squared_error
from math import sqrt
import markovify
id=0

############ load and digitize the data
data=pd.read_csv('/home/ebotian/MCM/tennis2.csv')

############

############# pre-process the data
    # fill nan with 0, and replace AD with 50
def pre_process(data):
    data = pd.get_dummies(data, columns=['winner_shot_type','serve_width','serve_depth','return_depth'])
    data = data.fillna(0)
    data = data.replace('AD', 50.0)
    #data['point_victor']=data["point_victor"].replace(2,0)
    #print(data.iloc[:,15])
    ###################################
    #for predict:
    conditions = [
        (data['server'] == 1) & (data['point_victor'] == 1),
        (data['server'] == 1) & (data['point_victor'] == 2),
        (data['server'] == 2) & (data['point_victor'] == 2),
        (data['server'] == 2) & (data['point_victor'] == 1)
    ]
    choices = ['P1_serve_win', 'P1_serve_lose', 'P2_serve_win', 'P2_serve_lose']
    data['state'] = np.select(conditions, choices, default='unknown')

    # split the data into different match
    grouped = dict(tuple(data.groupby(data['match_id'].ne(data['match_id'].shift()).cumsum())))

    # Rename the subdata
    subdata = {df['match_id'].iloc[0]: df for _, df in grouped.items()}

    # Create a new dataset from the first column, excluding duplicates
    match = pd.DataFrame(data.iloc[:, 0].drop_duplicates()).iloc[:,0].tolist()


    return subdata,match

subdata,match=pre_process(data)
#print(subdata[match[1]])
##############

def process_all_ids(subdata):
    features={}
    target={}
    for id in range(len(match)):
        #index_array = subdata[match[id]][subdata[match[id]]['server'] == 2].index.values
#        subdata[match[id]].loc[index_array, 'point_victor'] = 1 - subdata[match[id]].loc[index_array, 'point_victor']
        target[match[id]]=pd.DataFrame(subdata[match[id]]["point_victor"])
        # Add the "elapsed_time" column to the "target" DataFrame
        subdata[match[id]]['elapsed_time'] = pd.to_timedelta(subdata[match[id]]['elapsed_time'])
        #target[match[id]].insert(0, 'elapsed_time', subdata[match[id]]['elapsed_time'])
        #target['elapsed_time'] = target['elapsed_time'].dt.total_seconds()
        features[match[id]]=subdata[match[id]].drop(columns=["point_victor"]).iloc[:,4:]
        subdata[match[id]]=subdata[match[id]].drop(columns=["point_victor"])

    return target,features,subdata,index_array

# Replace with your actual match ids
target,features,subdata,index_array=process_all_ids(subdata)
#print(subdata[match[1]])
##############
##############
#invert the victor when the server is 2 to get server_victor
#and after the prediction, we invert the victor again
##############
# Calculate the time difference between consecutive rows
def get_average_interval(id, subdata):

    # Calculate the time difference between consecutive rows
    subdata[match[id]]['time_diff'] = subdata[match[id]]['elapsed_time'].diff()

    # Calculate the 5th and 95th percentiles
    lower_threshold = subdata[match[id]]['time_diff'].quantile(0.05)
    upper_threshold = subdata[match[id]]['time_diff'].quantile(0.95)

    # Exclude the top 5% and bottom 5% of periods
    filtered_diff = subdata[match[id]]['time_diff'][(subdata[match[id]]['time_diff'] > lower_threshold) & (subdata[match[id]]['time_diff'] < upper_threshold)]

    # Calculate the average of the remaining intervals
    average_interval = filtered_diff.mean()
    return average_interval.total_seconds()

average_interval_time=[]

for id in range(len(match)):
    average_interval_time.append(get_average_interval(id, subdata))

print(average_interval_time)

[43.78409, 40.59322, 39.725663, 37.31208, 39.0, 36.600706, 37.048543, 34.923076, 38.327956, 41.123674, 35.568493, 38.151639, 34.312252, 33.364197, 34.539772, 41.554794, 41.755, 41.395833, 34.773584, 34.866141, 40.021276, 35.890173, 31.554054, 37.925, 40.95238, 33.7, 37.491124, 42.31088, 37.605633, 47.819767, 47.070707]


In [35]:
############ add features
add_feature=[]
add_feature.append("p1_net_pt_probability")
add_feature.append("p2_net_pt_probability")
add_feature.append("p1_break_pt_probability")
add_feature.append("p2_break_pt_probability")


In [36]:
########## defining the new features
#for id in range(len(match)):
#  for cnt in range(len(add_feature)):
#    features[match[id]][add_feature[cnt]] = subdata[match[id]]['p1_net_pt']/ subdata[match[id]]["p1_net_pt_won"]
#    features[match[id]][add_feature[cnt]] = subdata[match[id]]['p2_net_pt']/ subdata[match[id]]["p2_net_pt_won"]
#    features[match[id]][add_feature[cnt]] = subdata[match[id]]["p1_break_pt"]/ subdata[match[id]]["p1_break_pt_won"]
#    features[match[id]][add_feature[cnt]] = subdata[match[id]]["p2_break_pt"]/ subdata[match[id]]["p2_break_pt_won"]


#data[add_feature_1] = data['p1_games'] - data['p2_games']
#data[add_feature_2] = data['p1_winner'] / (data['p1_winner'] + data['p2_winner'])
#data[add_feature_3] = data['p2_winner'] / (data['p1_winner'] + data['p2_winner'])
#data[add_feature_4] = data['p1_break_pt_won'] / (data['p1_break_pt_won'] + data['p1_break_pt_missed'])
#data[add_feature_5] = data['p2_break_pt_won'] / (data['p2_break_pt_won'] + data['p2_break_pt_missed'])


In [37]:
# split the data into features and target
#target=pd.DataFrame(subdata[match[id]]["point_victor"])
#
#subdata[match[id]]=subdata[match[id]].drop(columns=["point_victor"])
#features=subdata[match[id]].iloc[:,4:]
#print(features)

In [39]:
from sklearn.model_selection import train_test_split
# 数据划分
X_train={}
X_test={}
y_train={}
y_test={}
for id in range(len(match)):
  X_train[match[id]], X_test[match[id]], y_train[match[id]], y_test[match[id]] = train_test_split(features[match[id]], target[match[id]], test_size=0.3, random_state=42)

#print(X_train.isna().sum())

In [ ]:

# 数据标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 模型训练
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_scaled, y_train)

#print(X_test_scaled)

In [ ]:

# 模型预测
y_pred = clf.predict(X_test_scaled)

print(y_pred)

In [ ]:

# 模型评估
accuracy = accuracy_score(y_test, y_pred)
print(f'模型准确率: {accuracy}')

# 可以考虑输出特征重要性，帮助理解哪些因素对预测结果影响较大
importances = clf.feature_importances_
feature_names = X_train.columns
feature_importances = pd.Series(importances, index=feature_names).sort_values(ascending=False)
print(feature_importances)